In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd.variable import Variable
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import imageio
import os
from datetime import datetime
import numpy as np
from matplotlib import pyplot as plt
from data_utils.dataset import LSUNDataset

In [ ]:
# Root directory for dataset
dataroot = '../data/data'

# Number of workers for dataloader
workers = 1

# Batch size during training
batch_size = 100

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
#nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 128

# number of subsequent discriminator trainings
k = 1

# Number of training epochs
num_epochs = 2

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

# Learning rate for optimizers
lr = 2e-4

In [ ]:
# Create the dataset
dataset = LSUNDataset('../data/images.txt', dataroot,
    transform=transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize((0.5,),(0.5,))
    ]))

# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                        shuffle=True, num_workers=workers)

# to image transform
to_image = transforms.ToPILImage()

# Decide which device we want to run on
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        self.ngpu = ngpu
        super(Generator, self).__init__()
        self.n_features = nz
        self.n_out = 64*64*1

        self.fc0 = nn.Sequential(
            nn.Linear(self.n_features, 256),
            nn.LeakyReLU(0.2)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2)
        )
        self.fc3 = nn.Sequential(
            nn.Linear(1024, 2048),
            nn.LeakyReLU(0.2)
        )
        self.fc4 = nn.Sequential(
            nn.Linear(2048, self.n_out),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.fc0(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = x.view(-1, 1, 64, 64)
        return x

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.n_in = 64*64*1
        self.n_out = 1
        self.fc0 = nn.Sequential(
            nn.Linear(self.n_in, 2048),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.fc3 = nn.Sequential(
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.fc4 = nn.Sequential(
            nn.Linear(256, self.n_out),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x.view(-1, self.n_in)
        x = self.fc0(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        return x

In [ ]:
generator = Generator(ngpu)
discriminator = Discriminator(ngpu)

generator.to(device)
discriminator.to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    generator = nn.DataParallel(generator, list(range(ngpu)))
    discriminator = nn.DataParallel(discriminator, list(range(ngpu)))

g_optim = optim.Adam(generator.parameters(), lr=lr)
d_optim = optim.Adam(discriminator.parameters(), lr=lr)

In [ ]:
criterion = nn.BCELoss()

In [ ]:
def noise(n, n_features = nz):
    return Variable(torch.randn(n, n_features)).to(device)

def make_ones(size):
    return Variable(torch.ones(size, 1)).to(device)

def make_zeros(size):
    return Variable(torch.zeros(size, 1)).to(device)

In [ ]:
def train_discriminator(optimizer, real_data, fake_data):
    n = real_data.size(0)

    optimizer.zero_grad()

    prediction_real = discriminator(real_data)
    error_real = criterion(prediction_real, make_ones(n))
    error_real.backward()

    prediction_fake = discriminator(fake_data)
    error_fake = criterion(prediction_fake, make_zeros(n))
    error_fake.backward()

    optimizer.step()

    return error_real + error_fake

In [ ]:
def train_generator(optimizer, fake_data):
    n  = fake_data.size(0)

    optimizer.zero_grad()

    prediction = discriminator(fake_data)
    error = criterion(prediction, make_ones(n))

    error.backward()
    optimizer.step()

    return error

In [ ]:
test_noise = noise(64)
generator.train()
discriminator.train()

In [ ]:
g_losses = []
d_losses = []
images = []


for epoch in range(num_epochs):
    g_error = 0.0
    d_error = 0.0
    for i, data in enumerate(dataloader):
        imgs = data[0]
        n = len(imgs)
        for j in range(k):
            fake_data = generator(noise(n)).detach()
            real_data = imgs.to(device)
            d_error += train_discriminator(d_optim, real_data, fake_data)
        fake_data = generator(noise(n))
        g_error += train_generator(g_optim, fake_data)
        if i % 10 == 0:
            print('[%d/%d][%d/%d]\tLoss_Disscriminator: %.4f\tLoss_Generator: %.4f'
                  % (epoch, num_epochs, i, len(dataloader), (d_error/i).item(), (g_error/i).item()))

    img = generator(test_noise).cpu().detach()
    img = make_grid(img)
    images.append(img)
    g_losses.append((g_error/i).item())
    d_losses.append((d_error/i).item())
    print('Epoch {}: g_loss: {:.8f} d_loss: {:.8f}\r'.format(epoch, g_error/i, d_error/i))

print('Training Finished')

In [ ]:
plt.plot(g_losses, label='Generator_Losses')
plt.plot(d_losses, label='Discriminator Losses')
plt.legend()

In [ ]:
if not os.path.exists('../nets'):
    os.mkdir('../nets')
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
torch.save(generator.state_dict(), f'../nets/vanilla_netD_{format(timestamp)}')
torch.save(discriminator.state_dict(), f'../nets/vanilla_netG_{format(timestamp)}')

In [ ]:
if not os.path.exists('../prog_gifs'):
    os.mkdir('../prog_gifs')
imgs = [np.array(to_image(i)) for i in images]
imageio.mimsave(f'../prog_gifs/vanilla_progress_{format(timestamp)}.gif', imgs, duration=1)

In [ ]:
if not os.path.exists('../loss'):
    os.mkdir('../loss')
np.savetxt(f'../loss/vanilla_netD_{format(timestamp)}.txt', np.array(d_losses))
np.savetxt(f'../loss/vanilla_netG_{format(timestamp)}.txt', np.array(g_losses))